# APIs Lab
In this lab we will practice using APIs to retrieve and store data.

In [4]:
# Imports at the top
import json
import urllib
import pandas as pd
import numpy as np
import requests
import json
import re
import matplotlib.pyplot as plt
%matplotlib inline

## Exercise 1: IMDB Movies

Sometimes an API doesn't provide all the information we would like to get and we need to be creative.
Here we will use a combination of scraping and API calls to investigate the ratings and gross earnings of famous movies.

## 1.a Get top movies

The Internet Movie Database contains data about movies. Unfortunately it does not have a public API.

The page http://www.imdb.com/chart/top contains the list of the top 250 movies of all times. Retrieve the page using the requests library and then parse the html to obtain a list of the `movie_ids` for these movies. You can parse it with regular expression or using a library like `BeautifulSoup`.

**Hint:** movie_ids look like this: `tt2582802`

In [18]:
print "My link: http://mattbrown3255.herokuapp.com/site/imdb/top"

My link: http://mattbrown3255.herokuapp.com/site/imdb/top


In [2]:
# Get all the Movie IDs for the top 250 movies
from bs4 import BeautifulSoup
import urllib2

page_imdb = urllib2.urlopen('http://www.imdb.com/chart/top', 'lxml')
soup = BeautifulSoup(page_imdb.read())

tables_imdb = soup.findAll('tbody', attrs ={"class" : "lister-list"})
print len(tables_imdb)



/Users/Madballa55/anaconda/lib/python2.7/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 174 of the file /Users/Madballa55/anaconda/lib/python2.7/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


1


In [3]:
# Get all the Movie IDs for the top 250 movies

listy_movie_ids = []

for tid in tables_imdb[0].find_all('tr'):
    listy_movie_ids.append(tid.find('div', attrs ={"class" : "seen-widget"}).attrs['data-titleid'])

print len(listy_movie_ids)
print listy_movie_ids[0:5]


250
['tt0111161', 'tt0068646', 'tt0071562', 'tt0468569', 'tt0050083']



## 1.b Get top movies data

Although the Internet Movie Database does not have a public API, an open API exists at http://www.omdbapi.com.

Use this API to retrieve information about each of the 250 movies you have extracted in the previous step.
- Check the documentation of omdbapi.com to learn how to request movie data by id
- Define a function that returns a python object with all the information for a given id
- Iterate on all the IDs and store the results in a list of such objects
- Create a Pandas Dataframe from the list

In [54]:
# Check the documentation of omdbapi.com to learn how to request movie data by id
# http://www.omdbapi.com/?i=tt0111161&plot=short&r=json

list_of_dicty = []

for movie_id in listy_movie_ids:
    soup = BeautifulSoup(urllib2.urlopen('http://www.omdbapi.com/?i='+ movie_id + '&plot=short&r=json').read())
    parsed_json = json.loads(soup.text)
    
    list_of_dicty.append(parsed_json)

df_imdb = pd.DataFrame(list_of_dicty)

print len(df_imdb)
df_imdb.head()

250


,Actors,Awards,Country,Director,Genre,Language,Metascore,Plot,Poster,Rated,Released,Response,Runtime,Title,Type,Writer,Year,imdbID,imdbRating,imdbVotes
0,"Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",Nominated for 7 Oscars. Another 19 wins & 30 n...,USA,Frank Darabont,"Crime, Drama",English,80,Two imprisoned men bond over a number of years...,https://images-na.ssl-images-amazon.com/images...,R,14 Oct 1994,True,142 min,The Shawshank Redemption,movie,"Stephen King (short story ""Rita Hayworth and S...",1994,tt0111161,9.3,"1,725,904"
1,"Marlon Brando, Al Pacino, James Caan, Richard ...",Won 3 Oscars. Another 23 wins & 27 nominations.,USA,Francis Ford Coppola,"Crime, Drama","English, Italian, Latin",100,The aging patriarch of an organized crime dyna...,https://images-na.ssl-images-amazon.com/images...,R,24 Mar 1972,True,175 min,The Godfather,movie,"Mario Puzo (screenplay), Francis Ford Coppola ...",1972,tt0068646,9.2,"1,177,812"
2,"Al Pacino, Robert Duvall, Diane Keaton, Robert...",Won 6 Oscars. Another 10 wins & 20 nominations.,USA,Francis Ford Coppola,"Crime, Drama","English, Italian, Spanish, Latin, Sicilian",80,The early life and career of Vito Corleone in ...,https://images-na.ssl-images-amazon.com/images...,R,20 Dec 1974,True,202 min,The Godfather: Part II,movie,"Francis Ford Coppola (screenplay), Mario Puzo ...",1974,tt0071562,9.0,"807,734"
3,"Christian Bale, Heath Ledger, Aaron Eckhart, M...",Won 2 Oscars. Another 146 wins & 142 nominations.,"USA, UK",Christopher Nolan,"Action, Crime, Drama","English, Mandarin",82,N/A,https://images-na.ssl-images-amazon.com/images...,PG-13,18 Jul 2008,True,152 min,The Dark Knight,movie,"Jonathan Nolan (screenplay), Christopher Nolan...",2008,tt0468569,9.0,"1,711,711"
4,"Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",Nominated for 3 Oscars. Another 16 wins & 8 no...,USA,Sidney Lumet,"Crime, Drama",English,N/A,N/A,https://images-na.ssl-images-amazon.com/images...,APPROVED,01 Apr 1957,True,96 min,12 Angry Men,movie,"Reginald Rose (story), Reginald Rose (screenplay)",1957,tt0050083,8.9,"460,420"


In [5]:
# Define a function that returns a python object with all the information for a given id
def movie_data(movie_id):
    soup = BeautifulSoup(urllib2.urlopen('http://www.omdbapi.com/?i='+ movie_id + '&plot=short&r=json').read())
    parsed_json = json.loads(soup.text)
    
    print parsed_json

movie_data(movie_id='tt0111161')

{u'Plot': u'Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.', u'Rated': u'R', u'Response': u'True', u'Language': u'English', u'Title': u'The Shawshank Redemption', u'Country': u'USA', u'Writer': u'Stephen King (short story "Rita Hayworth and Shawshank Redemption"), Frank Darabont (screenplay)', u'Metascore': u'80', u'imdbRating': u'9.3', u'Director': u'Frank Darabont', u'Released': u'14 Oct 1994', u'Actors': u'Tim Robbins, Morgan Freeman, Bob Gunton, William Sadler', u'Year': u'1994', u'Genre': u'Crime, Drama', u'Awards': u'Nominated for 7 Oscars. Another 19 wins & 30 nominations.', u'Runtime': u'142 min', u'Type': u'movie', u'Poster': u'https://images-na.ssl-images-amazon.com/images/M/MV5BODU4MjU4NjIwNl5BMl5BanBnXkFtZTgwMDU2MjEyMDE@._V1_SX300.jpg', u'imdbVotes': u'1,725,904', u'imdbID': u'tt0111161'}


## 1.c Get gross data

The OMDB API is great, but it does not provide information about Gross Revenue of the movie. We'll revert back to scraping for this.

- Write a function that retrieves the gross revenue from the entry page at imdb.com
- The function should handle the exception of when the page doesn't report gross revenue
- Retrieve the gross revenue for each movie and store it in a separate dataframe

In [7]:
# Returns Gross Revenue Number
def gross_rev(movie_id):
    page_imdb_gross_rev = urllib2.urlopen('http://www.imdb.com/title/' + movie_id, 'lxml')
    soup = BeautifulSoup(page_imdb_gross_rev.read())
    gross = 0.0

    for a in soup.findAll('div', attrs ={"id" : "titleDetails"}):
        for b in a.findAll('div', attrs ={"class" : "txt-block"}):
            if 'Gross' in b.text:
                gross = float(b.text.split('$')[1].split(' ')[0].replace(',',''))
    
    return gross

gross_rev('tt0111161')

28341469.0

In [8]:
listy_gross_revenue = []

for movie_id in listy_movie_ids:
    listy_gross_revenue.append(gross_rev(movie_id))

print len(listy_gross_revenue)
print sum(listy_gross_revenue)
print listy_gross_revenue[0:5]


250
19179997972.0
[28341469.0, 134821952.0, 57300000.0, 533316061.0, 0.0]


## 1.d Data munging

- Now that you have movie information and gross revenue information, let's clean the two datasets.
- Check if there are null values. Be careful they may appear to be valid strings.
- Convert the columns to the appropriate formats. In particular handle:
    - Released
    - Runtime
    - year
    - imdbRating
    - imdbVotes
- Merge the data from the two datasets into a single one

In [55]:
# Merge the data from the two datasets into a single one
df_imdb['Gross_Revenue'] = listy_gross_revenue

In [70]:
# Check if there are null values. Be careful they may appear to be valid strings.
for x in df_imdb.columns.values:
    if sum(df_imdb[x].isnull()) > 0:
        print x, sum(df_imdb[x].isnull())
        
# Removing the 1 row that is N/A
df_imdb = df_imdb[df_imdb.Released != 'N/A']

# Convert the columns to the appropriate formats
df_imdb.Released = pd.to_datetime(df_imdb.Released, format='%d %b %Y')
df_imdb.Runtime = [float(x.split(' ')[0]) for x in df_imdb.Runtime]
df_imdb.Year = df_imdb.Year.astype(int)
df_imdb.imdbRating = df_imdb.imdbRating.astype(float)
df_imdb.imdbVotes = [float(x.replace(',','')) for x in df_imdb.imdbVotes]


df_imdb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 249 entries, 0 to 249
Data columns (total 21 columns):
Actors           249 non-null object
Awards           249 non-null object
Country          249 non-null object
Director         249 non-null object
Genre            249 non-null object
Language         249 non-null object
Metascore        249 non-null object
Plot             249 non-null object
Poster           249 non-null object
Rated            249 non-null object
Released         249 non-null datetime64[ns]
Response         249 non-null object
Runtime          249 non-null float64
Title            249 non-null object
Type             249 non-null object
Writer           249 non-null object
Year             249 non-null int64
imdbID           249 non-null object
imdbRating       249 non-null float64
imdbVotes        249 non-null float64
Gross_Revenue    249 non-null float64
dtypes: datetime64[ns](1), float64(4), int64(1), object(15)
memory usage: 42.8+ KB


## 1.e Text vectorization

There are several columns in the data that contain a comma separated list of items, for example the Genre column and the Actors column. Let's transform those to binary columns using the count vectorizer from scikit learn.

Append these columns to the merged dataframe.

**Hint:** In order to get the actors name right, you'll have to modify the `token_pattern` in the `CountVectorizer`.

In [72]:
# Create words in list into columns with dummy variables
from sklearn.feature_extraction.text import CountVectorizer

v = CountVectorizer(
    binary=True,  # Create binary features
    stop_words='english', # Ignore common words such as 'the', 'and'
    max_features=1000,
    ngram_range=(1, 2)
)
df_Plot = pd.DataFrame(v.fit_transform(df_imdb['Plot']).todense())
df_Plot.columns = v.get_feature_names()
df_Plot.head()

,000,1950s,1984,24,abandoned,able,accident,accidentally,action,actress,...,wrong,wwii,year,year old,years,york,young,young boy,young man,young woman
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [73]:
# Create words in list into columns with dummy variables
from sklearn.feature_extraction.text import CountVectorizer

v = CountVectorizer(
    binary=True,  # Create binary features
    stop_words='english', # Ignore common words such as 'the', 'and'
    max_features=1000,
    ngram_range=(1, 2)
)
df_Genre = pd.DataFrame(v.fit_transform(df_imdb['Genre']).todense())
df_Genre.columns = v.get_feature_names()
df_Genre.head()

,action,action adventure,action biography,action crime,action drama,action mystery,action sci,action thriller,adventure,adventure biography,...,noir thriller,romance,romance thriller,romance war,sci,sci fi,sport,thriller,war,western
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [74]:
# Create words in list into columns with dummy variables
from sklearn.feature_extraction.text import CountVectorizer

v = CountVectorizer(
    binary=True,  # Create binary features
    stop_words='english', # Ignore common words such as 'the', 'and'
    max_features=1000,
    ngram_range=(1, 2)
)
df_Actors = pd.DataFrame(v.fit_transform(df_imdb['Actors']).todense())
df_Actors.columns = v.get_feature_names()
df_Actors.head()

,aamir,aamir khan,abraham,adam,adrien,adrien brody,affleck,al,al pacino,alan,...,zacapa,zacapa brad,zach,zach grenier,zebrowski,zebrowski ed,zoe,zoe saldana,álvaro,álvaro guerrero
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [77]:
# Append these columns to the merged dataframe.
df_imdb_merged = pd.concat([df_imdb, df_Actors, df_Genre, df_Plot], axis=1, join_axes=[df_imdb.index])

print len(df_imdb_merged.columns.values)
print "Whoa"


2112
Whoa


## Exercise 2: Showing Our Data

We did all that hard work. Let's show it to the world! First save your .csv file.

```
df.to_csv('imdb.csv', sep='\t', encoding='utf-8')
```

In [2]:
df_imdb_merged.to_csv('imdb.csv', sep='\t', encoding='utf-8')

NameError: name 'df_imdb_merged' is not defined

## 2.a Add model

Add a new model to the models.py file in your website folder. It should contain each movie's meta data such as rank, summary, genre, and rating. Here is an example of a movies model.

```
class TopMovies(models.Model):
    name = models.CharField(max_length=200)
    rank = models.IntegerField(default=-1)
    created = models.DateTimeField(auto_now_add=True, blank=True)
```

## 2.b Add Url
Add the following path to your url.py file.

```
url(r'^imdb/top/$', views.imdb, name='imdb')
```

## 2.c Create View

Create your view. Remember you need to name it imdb! Your view should contain the following logic:

- If your imdb database has no values in it
    - Open up your imdb.csv file and populate the model
- Retrieve model and show the data on your webpage

## 2.d Create Template

Link a new template to your view. Your template must do the following things:

- Use a table, rows, and columns to organize your data
- Have a different background color besides white
- Show your image poster links as images

## 2.e Submit Url

Paste your new url below

In [3]:
df_imdb_merged.head()

NameError: name 'df_imdb_merged' is not defined

In [16]:
df = pd.read_csv('imdb.csv', sep='\t')

df.rank

<bound method DataFrame.rank of      Unnamed: 0                                             Actors  \
0             0  Tim Robbins, Morgan Freeman, Bob Gunton, Willi...   
1             1  Marlon Brando, Al Pacino, James Caan, Richard ...   
2             2  Al Pacino, Robert Duvall, Diane Keaton, Robert...   
3             3  Christian Bale, Heath Ledger, Aaron Eckhart, M...   
4             4  Martin Balsam, John Fiedler, Lee J. Cobb, E.G....   
5             5  Liam Neeson, Ben Kingsley, Ralph Fiennes, Caro...   
6             6  Tim Roth, Amanda Plummer, Laura Lovelace, John...   
7             7   Noel Appleby, Ali Astin, Sean Astin, David Aston   
8             8  Eli Wallach, Clint Eastwood, Lee Van Cleef, Al...   
9             9  Edward Norton, Brad Pitt, Meat Loaf, Zach Grenier   
10           10  Alan Howard, Noel Appleby, Sean Astin, Sala Baker   
11           11  Mark Hamill, Harrison Ford, Carrie Fisher, Bil...   
12           12  Tom Hanks, Rebecca Williams, Sally Field,

In [8]:
df.info(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 249 entries, 0 to 248
Data columns (total 2113 columns):
Unnamed: 0                   int64
Actors                       object
Awards                       object
Country                      object
Director                     object
Genre                        object
Language                     object
Metascore                    float64
Plot                         object
Poster                       object
Rated                        object
Released                     object
Response                     bool
Runtime                      float64
Title                        object
Type                         object
Writer                       object
Year                         int64
imdbID                       object
imdbRating                   float64
imdbVotes                    float64
Gross_Revenue                float64
aamir                        float64
aamir khan                   float64
abraham                   